In [ ]:
import urllib.request
from  bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions
from selenium.common.exceptions import NoSuchElementException
import selenium
import time
import pandas as pd

user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
def url_open(url):
    request = urllib.request.Request(url,headers={'User-Agent': user_agent})
    response = urllib.request.urlopen(request)
    html = response.read()
    soup_1 = BeautifulSoup(html,"lxml")
    return soup_1

available_result = []
colour = []
driver = selenium.webdriver.Chrome(executable_path="D:\chromedriver.exe")
url = "https://www.amazon.in/Jockey-2714-0105-BLACK-Black-XL-2714-0105-BLACK_Black_XL/dp/B071YSSRDR/"
soup = url_open(url)
title = soup.find("span",attrs = {"id":"productTitle"}).get_text().strip()
for i in soup.find("ul",attrs = {"class":"a-unordered-list a-nostyle a-button-list a-declarative a-button-toggle-group a-horizontal a-spacing-top-micro swatches swatchesRec\
tangle imageSwatches"}).find_all('li'):
    colour.append(i['data-defaultasin'])

for col in colour:
    url = "https://www.amazon.in/Jockey-2714-0105-BLACK-Black-XL-2714-0105-BLACK_Black_XL/dp/{}".format(col)
    print(url)
    soup_1 = url_open(url)
    img_link = []
    for i in soup_1.find_all("span",attrs = {"class":"a-button-text"}):
        img = i.find('img')
        if img:
            img_link.append(img['src'])

    img_link = img_link[:-1]
    available = []
    colour_name = soup_1.find("span",attrs ={"class":'selection'}).get_text().strip()
    for i in soup_1.find_all('select',attrs={"id":"native_dropdown_selected_size_name"})[0].find_all('option')[1:]:
        if i['class'][0] == "dropdownAvailable":
            available.append([title,url,img_link,colour_name ,i.get_text().strip(),"available"])
        if i['class'][0] == "dropdownUnavailable":
            available.append([title,url,img_link,colour_name ,i.get_text().strip(),'not available'])
    driver.get(url)
    total_sizes = len(soup_1.find_all('select',attrs={"id":"native_dropdown_selected_size_name"})[0].find_all('option'))
    #print(total_sizes)
    for i in range(1,total_sizes):
        obj = Select(driver.find_element_by_id('native_dropdown_selected_size_name'))
        obj.select_by_index(i)
        time.sleep(2)
        html = driver.page_source
        soup_1 = BeautifulSoup(html,"lxml")
        #print(available[i-1])
        try:
            if available[i-1][-1] == 'available':
                txt = soup_1.find("span",attrs ={"id":'priceblock_ourprice'}).get_text().strip().split()[1]
            else:
                txt = ''
        except:
             txt = ''   
        available[i-1].append(txt)
    available_result += available.copy()
available_result